In [ ]:
# @title Docker Firefox
from threading import Timer
from IPython.display import HTML, clear_output
from subprocess import Popen, PIPE
from tqdm import tqdm
pbar = tqdm(range(10))


!pip install udocker  > /dev/null
pbar.update(2)
pbar.refresh()
!udocker --allow-root install > /dev/null
pbar.update(3)
pbar.refresh()
!udocker --allow-root pull jlesage/firefox > /dev/null #consol/debian-xfce-vnc > /dev/null #
pbar.update(6)
pbar.refresh()
!udocker --allow-root create --name=firefox jlesage/firefox > /dev/null
pbar.update(7)
pbar.refresh()
!ssh-keygen -t rsa -N '' -f ~/.ssh/id_rsa <<< yes > /dev/null
pbar.update(8)
pbar.refresh()

def runTunnel():
  clear_output()
  !ssh srv.us -R1:localhost:5900 -o StrictHostKeyChecking=no

t=Timer(5,runTunnel)
t.start()
!udocker --allow-root  run -p 5900:5900 --volume=/content:/config/downloads firefox > /dev/null
pbar.update(10)
pbar.refresh()